############# SHOCK ADAPTATION FOOD SUPPLY MODEL #############


 SOPHIA BAUM - 2024

In [1]:
tau = 10                            # number of iterations
compensation=True                 # turn adaptation on
limit_abs_sim=1000                  # event limits
limit_rel_sim=0.26
limit_dev_sim=0.32

In [2]:
### IMPORT ###
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
from pandas import IndexSlice as idx

In [4]:
import scipy.io as io
import scipy.sparse as sprs

In [5]:
import numpy as np

In [6]:
import os
print(os.getcwd())

/Users/valentinbast/uni/compelx_sys_modelling/project/adaptive_food_supply_network-main/csm_project_


## PARAMETERS ###

In [7]:
input_folder = 'input/'                 # folder with parameters
output_folder =  'results/'             # folder to write results to
losses = 'evaluation/'                  # folder to store the refined results to

In [8]:
a_shock = 'IND'

In [9]:
a_frame = pd.read_csv(input_folder+'a_frame.csv')
area_value = a_frame.loc[a_frame['code'] == a_shock, 'code'].values[0]

## LOADING DATA ###

In [10]:
# Load information
io_codes = pd.read_csv(input_folder+'io_codes_alph.csv').drop('Unnamed: 0', axis = 1)
su_codes = pd.read_csv(input_folder+'su_codes_alph.csv').drop('Unnamed: 0', axis = 1)

In [11]:
# Create single indexes
areas = np.array(sorted(set(io_codes['area'])))
items = np.array(sorted(set(io_codes['item'])))
processes = np.array(sorted(set(su_codes['proc'])))

In [12]:
# Create multi indexes
ai_index = pd.MultiIndex.from_product([areas, items])
ap_index = pd.MultiIndex.from_product([areas, items])

In [13]:
# Load  further information on countries
a_frame = pd.read_csv(input_folder+'a_frame.csv')

In [14]:
# Load the result of the shocked simulation
X = pd.read_csv(output_folder+'base.csv', index_col=[0,1], header=[0])
XS_comp = pd.read_csv(output_folder + area_value + '_comp.csv', index_col=[0, 1], header=[0, 1])
XS_no_comp = pd.read_csv(output_folder+area_value + '_no_comp.csv', index_col=[0,1], header=[0,1])

In [15]:
# Flatten multi-level columns
XS_comp.columns = ['_'.join(col).strip() for col in XS_comp.columns.values]
XS_no_comp.columns = ['_'.join(col).strip() for col in XS_no_comp.columns.values]

# Inspect the flattened DataFrame
print(XS_comp.head())
print(XS_no_comp.head())

                                   India_Abaca  India_Alcohol, Non-Food  \
area        item                                                          
Afghanistan Abaca                     0.000000                 0.000000   
            Alcohol, Non-Food      2025.485805              1747.200753   
            Apples and products  273405.028019            273405.028019   
            Asses                     0.000000                 0.000000   
            Bananas               97000.991410             97000.991410   

                                 India_Apples and products    India_Asses  \
area        item                                                            
Afghanistan Abaca                                 0.000000       0.000000   
            Alcohol, Non-Food                  2025.387167    2025.485805   
            Apples and products              273384.140065  273405.028019   
            Asses                                 0.000000       0.000000   
            

## COMPUTATIONS ###

In [16]:
# Compute relative loss
RL_no_comp = XS_no_comp.copy()
RL_comp = XS_comp.copy()
for col in XS_no_comp.columns:
    RL_no_comp[col] = ((X['base'] - RL_no_comp[col])/X['base']).fillna(0)
    RL_comp[col] = ((X['base'] - RL_comp[col])/X['base']).fillna(0)  
RL_no_comp[RL_no_comp < -1] = -1
RL_comp[RL_comp < -1] = -1

In [17]:
# Setup a dataframe for the relative loss
RL_no_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
RL_no_comp.columns.names = ['a_shock','i_shock']
RL_no_comp.index.names = ['a_receive','i_receive'] 
RL_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
RL_comp.columns.names = ['a_shock','i_shock']
RL_comp.index.names = ['a_receive','i_receive'] 

In [18]:
# Save
RL_no_comp.to_csv(losses+'RL-'+ area_value+'_no_comp.csv')
RL_comp.to_csv(losses+'RL-'+ area_value +'_comp.csv')

In [19]:
# Compute absolute loss
AL_no_comp = XS_no_comp.copy()
AL_comp = XS_comp.copy()
for col in XS_no_comp.columns:
    AL_no_comp[col] = X['base'] - XS_no_comp[col]
    AL_comp[col] = X['base'] - XS_comp[col]

In [20]:
# Setup a dataframe for the absolute loss
AL_no_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_no_comp.columns.names = ['a_shock','i_shock']
AL_no_comp.index.names = ['a_receive','i_receive'] 
AL_comp.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_comp.columns.names = ['a_shock','i_shock']
AL_comp.index.names = ['a_receive','i_receive']  

In [21]:
# Save
AL_no_comp.to_csv(losses+'AL-'+ area_value +'_no_comp.csv')
AL_comp.to_csv(losses+'AL-'+ area_value +'_comp.csv')

In [22]:
# Ensure MultiIndex for AL_no_comp and AL_comp
AL_no_comp.index.names = ['area', 'item']
AL_comp.index.names = ['area', 'item']

In [23]:
# Step 1: Reset index
AL_no_comp_reset = AL_no_comp.reset_index()
AL_comp_reset = AL_comp.reset_index()

# Step 2: Flatten MultiIndex columns if needed
if isinstance(AL_no_comp_reset.columns, pd.MultiIndex):
    AL_no_comp_reset.columns = ['_'.join(filter(None, map(str, col))) for col in AL_no_comp_reset.columns]

if isinstance(AL_comp_reset.columns, pd.MultiIndex):
    AL_comp_reset.columns = ['_'.join(filter(None, map(str, col))) for col in AL_comp_reset.columns]

# Step 3: Merge using correct column
if 'area' in AL_no_comp_reset.columns and 'area' in a_frame.columns:
    AL_no_comp_pc = AL_no_comp_reset.merge(
        a_frame[['area', 'population']], left_on='area', right_on='area', how='left'
    )
    AL_comp_pc = AL_comp_reset.merge(
        a_frame[['area', 'population']], left_on='area', right_on='area', how='left'
    )
else:
    raise KeyError("Missing 'a_receive' in AL_* or 'area' in a_frame.")

In [24]:
# Step 1: Flatten columns if MultiIndex
if isinstance(AL_no_comp_pc.columns, pd.MultiIndex):
    AL_no_comp_pc.columns = ['_'.join(filter(None, map(str, col))) for col in AL_no_comp_pc.columns]
if isinstance(AL_comp_pc.columns, pd.MultiIndex):
    AL_comp_pc.columns = ['_'.join(filter(None, map(str, col))) for col in AL_comp_pc.columns]

# Step 2: Compute absolute loss per capita
# This excludes non-numeric or unrelated columns
for col in AL_no_comp.columns:
    col_flat = '_'.join(filter(None, map(str, col))) if isinstance(col, tuple) else col
    if col_flat in AL_no_comp_pc.columns:
        AL_no_comp_pc[col_flat] = AL_no_comp_pc[col_flat] / AL_no_comp_pc['population']
        AL_comp_pc[col_flat] = AL_comp_pc[col_flat] / AL_comp_pc['population']

# Step 3: Set index and scale by 1000
AL_no_comp_pc = (
    AL_no_comp_pc
    .set_index(['area', 'item'])  # assumes 'item' is available after reset_index
    .drop(columns='population')
    * 1000
)

AL_comp_pc = (
    AL_comp_pc
    .set_index(['area', 'item'])
    .drop(columns='population')
    * 1000
)

In [25]:
# Setup a dataframe for the absolute loss
AL_no_comp_pc.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_no_comp_pc.columns.names = ['a_shock','i_shock']
AL_no_comp_pc.index.names = ['a_receive','i_receive'] 
AL_no_comp_pc.columns = pd.MultiIndex.from_product([[a_shock],items])
AL_no_comp_pc.columns.names = ['a_shock','i_shock']
AL_no_comp_pc.index.names = ['a_receive','i_receive']  

In [26]:
# Save
AL_no_comp_pc.to_csv(losses+'AL_pc-'+ a_shock +'_no_comp.csv')
AL_no_comp_pc.to_csv(losses+'AL_pc-'+ a_shock +'_comp.csv')